In [444]:
import pandas as pd
import pytz
#import data
chart_df = pd.read_csv('C:/Users/clair/OneDrive/Documents/Emory/Career/CLIFProject/chartevents4.csv')
mappings = pd.read_csv('C:/Users/clair/OneDrive/Documents/Emory/Career/CLIFProject/mappings.csv')

#make sure they read in correctly
#print(chart_df.head)
#print(mappings.head)

In [446]:
#print column names
print(chart_df.columns)
print(mappings.columns)

#important variables from https://clif-consortium.github.io/website/data-dictionary.html#vitals
#  hospitalization_id -> hadm_id
#  recorded_dttm -> charttime
#  vital_name -> label...vital_name
#  vital_category
#  vital_value -> valuenum
#  meas_site_name
#important to note that valueuom has units
#need itemid to merge

#there's a good bit of information in mappings that we don't need, let's narrow the dataset
mappings_thin = mappings[['itemid', 'vital_category', 'label = vital_name', 'meas_site_name']]

Index(['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime',
       'storetime', 'itemid', 'value', 'valuenum', 'valueuom', 'warning'],
      dtype='object')
Index(['vital_category', 'meas_site_name', 'itemid', 'label = vital_name',
       'abbreviation', 'linksto', 'category', 'unitname', 'param_type',
       'count', 'value_instances', 'note / comment', 'status', 'Unnamed: 13',
       'Temperature Site', 'meas_site_name.1'],
      dtype='object')


In [448]:
#now let's merge mappings and chart_df by itemid using an inner join
merged_data = pd.merge(chart_df, mappings, on = 'itemid', how = 'inner')

#check and make sure that went well
print("chart_df:", chart_df.shape)
print("mappings:", mappings.shape)
print("merged_data:", merged_data.shape)
#looks good!

chart_df: (668862, 11)
mappings: (61, 16)
merged_data: (125770, 26)


In [450]:
#let's investigate our variables
merged_data.info()
#let's only keep the necessary variables
merged_data = merged_data[['itemid', 'hadm_id', 'charttime', 'label = vital_name', 'vital_category', 'valuenum', 'valueuom', 'meas_site_name']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125770 entries, 0 to 125769
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   subject_id          125770 non-null  int64  
 1   hadm_id             125770 non-null  int64  
 2   stay_id             125770 non-null  int64  
 3   caregiver_id        125770 non-null  float64
 4   charttime           125770 non-null  object 
 5   storetime           125770 non-null  object 
 6   itemid              125770 non-null  int64  
 7   value               125770 non-null  object 
 8   valuenum            97031 non-null   float64
 9   valueuom            96795 non-null   object 
 10  warning             125770 non-null  float64
 11  vital_category      125770 non-null  object 
 12  meas_site_name      124822 non-null  object 
 13  label = vital_name  125770 non-null  object 
 14  abbreviation        125770 non-null  object 
 15  linksto             8512 non-null 

In [452]:
#let's look at our variables again
merged_data.info()
#looks good!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125770 entries, 0 to 125769
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   itemid              125770 non-null  int64  
 1   hadm_id             125770 non-null  int64  
 2   charttime           125770 non-null  object 
 3   label = vital_name  125770 non-null  object 
 4   vital_category      125770 non-null  object 
 5   valuenum            97031 non-null   float64
 6   valueuom            96795 non-null   object 
 7   meas_site_name      124822 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 7.7+ MB


In [454]:
#let's change the names of our variables
merged_data = merged_data.rename(columns = {'hadm_id' : 'hospitalization_id', 'charttime' : 'recorded_dttm', 'label = vital_name' : 'vital_name', 
                                            'valuenum' : 'vital_value'})
merged_data.info()
#looks good!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125770 entries, 0 to 125769
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   itemid              125770 non-null  int64  
 1   hospitalization_id  125770 non-null  int64  
 2   recorded_dttm       125770 non-null  object 
 3   vital_name          125770 non-null  object 
 4   vital_category      125770 non-null  object 
 5   vital_value         97031 non-null   float64
 6   valueuom            96795 non-null   object 
 7   meas_site_name      124822 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 7.7+ MB


In [456]:
#now let's deal with our missing values
merged_data.head()
#according to the guide, we only have the following vital_categories: temp_c, heart_rate, sbp, dbp, spo2, respiratory_rate, map, height_cm, weight_kg

,itemid,hospitalization_id,recorded_dttm,vital_name,vital_category,vital_value,valueuom,meas_site_name
0,223769,20626031,12/16/2132 0:00,O2 Saturation Pulseoxymetry Alarm - High,NO MAPPING,100.0,%,NO MAPPING
1,223935,20626031,12/16/2132 0:00,PostTib. Pulses R,NO MAPPING,NaN,NaN,NO MAPPING
2,220048,20626031,12/16/2132 0:00,Heart Rhythm,NO MAPPING,NaN,NaN,NO MAPPING
3,223943,20626031,12/16/2132 0:00,Dorsal PedPulse L,NO MAPPING,NaN,NaN,NO MAPPING
4,220060,20626031,12/16/2132 0:00,Pulmonary Artery Pressure diastolic,NO MAPPING,12.0,mmHg,NO MAPPING


In [458]:
#let's look at vital_categories
print(merged_data['vital_category'].value_counts())
#we need to remove all rows with NO MAPPING 
merged_data = merged_data[merged_data['vital_category'] != "NO MAPPING"]

vital_category
NO MAPPING          29394
map                 14390
dbp                 14367
sbp                 14366
respiratory_rate    13913
heart_rate          13913
spo2                13540
temp_c               7564
MAPPED ELSEWHERE     3446
weight_kg             806
height_cm              71
Name: count, dtype: int64


In [460]:
#we need to remove all rows with MAPPED ELSEWHERE
merged_data = merged_data[merged_data['vital_category'] != "MAPPED ELSEWHERE"]
#let's make sure that worked properly
print(merged_data['vital_category'].value_counts())
#looks good!

vital_category
map                 14390
dbp                 14367
sbp                 14366
respiratory_rate    13913
heart_rate          13913
spo2                13540
temp_c               7564
weight_kg             806
height_cm              71
Name: count, dtype: int64


In [462]:
#another restriction we have has to do with units (in valueuom column)
#all vital_category = temp_c must have Celsius as the unit
#let's investigate temp_c rows
temp_c = merged_data[merged_data['vital_category'] == 'temp_c']
print(temp_c['valueuom'].value_counts())

valueuom
NaN    3794
°F     3379
°C      391
Name: count, dtype: int64


In [406]:
#okay so we need to remove rows with °F
merged_data = merged_data[merged_data['valueuom'] != '°F']
#now let's make sure all vital_category = height_cm is in centimeters
height_cm = merged_data[merged_data['vital_category'] == 'height_cm']
print(height_cm['valueuom'].value_counts())
#looks good!

valueuom
cm    71
Name: count, dtype: int64


In [442]:
#now let's make sure all vital_category = weight_kg is in kg
weight_kg = merged_data[merged_data['vital_category'] == 'weight_kg']
print(weight_kg['valueuom'].value_counts())
#looks good!

valueuom
kg    570
Name: count, dtype: int64


In [410]:
#now let's make sure all vital_category = map is in mm/Hg
vc_map = merged_data[merged_data['vital_category'] == 'map']
print(vc_map['valueuom'].value_counts())
#looks good!

valueuom
mmHg    14390
Name: count, dtype: int64


In [412]:
#now let's make sure all vital_category = spo2 is in %
spo2 = merged_data[merged_data['vital_category'] == 'spo2']
print(spo2['valueuom'].value_counts())
#looks good!

valueuom
%    13540
Name: count, dtype: int64


In [414]:
#let's see how our data looks now
merged_data.info()
#we still have some missingness in the vital_value column
#let's investigate that
missing_data = merged_data[merged_data['vital_value'].isna()]
missing_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 89551 entries, 6 to 125768
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   itemid              89551 non-null  int64  
 1   hospitalization_id  89551 non-null  int64  
 2   recorded_dttm       89551 non-null  object 
 3   vital_name          89551 non-null  object 
 4   vital_category      89551 non-null  object 
 5   vital_value         85757 non-null  float64
 6   valueuom            85521 non-null  object 
 7   meas_site_name      88674 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 6.1+ MB


,itemid,hospitalization_id,recorded_dttm,vital_name,vital_category,vital_value,valueuom,meas_site_name
11,224642,20626031,12/16/2132 0:00,Temperature Site,temp_c,NaN,NaN,SPECIAL CASE
22,224642,20626031,12/16/2132 1:00,Temperature Site,temp_c,NaN,NaN,SPECIAL CASE
29,224642,20626031,12/16/2132 2:00,Temperature Site,temp_c,NaN,NaN,SPECIAL CASE
49,224642,20626031,12/16/2132 3:00,Temperature Site,temp_c,NaN,NaN,SPECIAL CASE
51,224642,20626031,12/16/2132 4:00,Temperature Site,temp_c,NaN,NaN,SPECIAL CASE


In [416]:
#let's look at the rows with meas_site_name = SPECIAL CASE
print(merged_data['meas_site_name'].value_counts())
special_case = merged_data[merged_data['meas_site_name'] == 'SPECIAL CASE']
special_case.head()
#it appears we need to remove the temp_c rows with NaN in the valueuom column
merged_data = merged_data[~((merged_data['vital_category'] == 'temp_c') & (merged_data['valueuom'] != '°C'))]

meas_site_name
not specified    66420
arterial         18069
SPECIAL CASE      4185
Name: count, dtype: int64


In [418]:
#let's check height to make sure eveything is cm, no NaN
height = merged_data[merged_data['vital_category'] == 'height_cm']
height.head()
height['valueuom'].isna().any()
#looks good!

False

In [420]:
#let's check weight to make sure eveything is kg, no NaN
weight = merged_data[merged_data['vital_category'] == 'weight_kg']
weight.head()
#let's remove any weight_kg rows without kg as the unit
merged_data = merged_data[~((merged_data['vital_category'] == 'weight_kg') & (merged_data['valueuom'] != 'kg'))]

In [422]:
#let's check map to make sure eveything is mmHg, no NaN
c_map = merged_data[merged_data['vital_category'] == 'map']
c_map['valueuom'].isna().any()
#looks good!

False

In [424]:
#let's check spo2 to make sure eveything is %, no NaN
spo = merged_data[merged_data['vital_category'] == 'spo2']
spo['valueuom'].isna().any()
#looks good!

False

In [426]:
#let's check our missing now
merged_data.info()
#yay now there's no missing in the vital_value column
#we don't need valueuom, and some rows don't require units anyways
#meas_site_name is optional, so that missingness is okay

<class 'pandas.core.frame.DataFrame'>
Index: 85521 entries, 6 to 125768
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   itemid              85521 non-null  int64  
 1   hospitalization_id  85521 non-null  int64  
 2   recorded_dttm       85521 non-null  object 
 3   vital_name          85521 non-null  object 
 4   vital_category      85521 non-null  object 
 5   vital_value         85521 non-null  float64
 6   valueuom            85521 non-null  object 
 7   meas_site_name      84880 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 5.9+ MB


In [428]:
#now let's make sure we have the correct data types
merged_data.dtypes
#we need to make hosp_id into a string
merged_data['hospitalization_id'] = merged_data['hospitalization_id'].astype(str)
#we need to make recorded_dttm into datetime UTC
merged_data['recorded_dttm'] = pd.to_datetime(merged_data['recorded_dttm'])
merged_data['recorded_dttm'] = merged_data['recorded_dttm'].dt.tz_localize('UTC')

In [430]:
#let's check the data types now
merged_data.dtypes
#looks good!

itemid                              int64
hospitalization_id                 object
recorded_dttm         datetime64[ns, UTC]
vital_name                         object
vital_category                     object
vital_value                       float64
valueuom                           object
meas_site_name                     object
dtype: object

In [432]:
merged_data.head()

,itemid,hospitalization_id,recorded_dttm,vital_name,vital_category,vital_value,valueuom,meas_site_name
6,220210,20626031,2132-12-16 00:00:00+00:00,Respiratory Rate,respiratory_rate,19.0,insp/min,not specified
7,220051,20626031,2132-12-16 00:00:00+00:00,Arterial Blood Pressure diastolic,dbp,37.0,mmHg,arterial
9,223762,20626031,2132-12-16 00:00:00+00:00,Temperature Celsius,temp_c,37.0,°C,SPECIAL CASE
12,220052,20626031,2132-12-16 00:00:00+00:00,Arterial Blood Pressure mean,map,58.0,mmHg,arterial
13,220045,20626031,2132-12-16 00:00:00+00:00,Heart Rate,heart_rate,80.0,bpm,not specified


In [434]:
#we don't need itemid or valueuom in our final dataset, so let's remove those
df_final = merged_data[['hospitalization_id', 'recorded_dttm', 'vital_name', 'vital_category', 'vital_value', 'meas_site_name']]
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85521 entries, 6 to 125768
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   hospitalization_id  85521 non-null  object             
 1   recorded_dttm       85521 non-null  datetime64[ns, UTC]
 2   vital_name          85521 non-null  object             
 3   vital_category      85521 non-null  object             
 4   vital_value         85521 non-null  float64            
 5   meas_site_name      84880 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 4.6+ MB


In [464]:
#export dataframe
df_final.to_csv('final_formatted_df.csv', index = False)